In [1]:
# Load Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy import stats

from multiprocessing import cpu_count

# Fill Missing Values

Most machine learning models do not allow for missing values in the input data.

In categorical variables, missing values can be transformed into a new category 'Missing', solving this way the issue. They can also be replaced with the mode.

For numerical variables, these missing values can be filled using different approaches.

## Load Data

In [2]:
dat = pd.read_csv('../data/userbase.csv', sep = ";")
dat

,user,booking_date,origin_airport,price,sales channel,company,user_country,index
0,user5,2018-11-01,MAD,58.200001,online,I2,NaN,1
1,user7,2018-11-01,DUB,147.500000,online,I2,NaN,2
2,user4,2018-11-02,TFS,24.049999,online,I2,NaN,3
3,user8,2018-10-29,MAD,59.709999,online,I2,NaN,4
4,user7,2018-11-01,LPA,37.299999,call center,I2,NaN,5
...,...,...,...,...,...,...,...,...
995,user2,2018-11-01,JMK,99.849998,online,I2,NaN,996
996,user10,2018-11-01,SVQ,34.610001,online,I2,NaN,997
997,user4,2018-10-30,MAD,49.880001,online,I2,NaN,998
998,user10,2018-11-02,CDG,152.960007,online,I2,NaN,999


## Detect Missing Values

Let's count the number of missing values of each variable of the dataset.

In [28]:
dat.apply(lambda x: 100*np.sum(x.isna())/len(x))

user               0.0
booking_date       0.0
origin_airport     0.0
price              1.4
sales channel      0.0
company            0.0
user_country      99.0
index              0.0
dtype: float64

price and user_country have missing values. We should try to fill them.

## Approach 0: Remove rows with missing values.

The most basic approach would be to simply remove any row with missing values. However, although this approach is sometimes recommended in courses and tutorial, it is not advisable in real life. Doing this will probably leave you with almost no data ;p.

In [4]:
dat_new = dat.copy()
dat_new.dropna(axis = 0, how = 'any', inplace = True)
dat_new

,user,booking_date,origin_airport,price,sales channel,company,user_country,index
152,user3,2018-10-30,MAD,44.320000,travel agency,I2,Spain,153
165,user6,2018-11-02,MAD,41.119999,travel agency,I2,Spain,166
356,user3,2018-10-31,MAD,40.209999,travel agency,I2,Spain,357
382,user8,2018-11-01,MAD,87.000000,online,I2,Spain,383
523,user3,2018-10-31,LPA,7.440000,travel agency,I2,Spain,524
552,user2,2018-11-01,MAH,48.349998,online,I2,Spain,553
584,user3,2018-11-02,SVQ,96.540001,online,I2,Spain,585
642,user8,2018-11-01,VGO,84.160004,online,I2,Spain,643
656,user6,2018-11-01,LPA,39.320000,online,I2,Spain,657
706,user3,2018-11-01,MAD,17.610001,travel agency,I2,Spain,707


Doesn't seem to be a good idea...

## Categorical

In [5]:
dat.dtypes

user               object
booking_date       object
origin_airport     object
price             float64
sales channel      object
company            object
user_country       object
index               int64
dtype: object

In [6]:
categorical_variables = dat.columns.values[dat.dtypes == 'object'].tolist()
numerical_variables = dat.columns.values[dat.dtypes == 'float64'].tolist()

In [7]:
categorical_variables

['user',
 'booking_date',
 'origin_airport',
 'sales channel',
 'company',
 'user_country']

In [8]:
numerical_variables

['price']

### Approach 1: Replace with mode.

The next most basic approach regarding categorical variables will be to replace missing values with most frequent category or mode.

In [10]:
modes = dat[categorical_variables].mode()

In [11]:
modes

,user,booking_date,origin_airport,sales channel,company,user_country
0,user9,2018-11-01,MAD,online,I2,Spain


Let's use this mode values to replace NAs.

In [12]:
dat

,user,booking_date,origin_airport,price,sales channel,company,user_country,index
0,user5,2018-11-01,MAD,58.200001,online,I2,NaN,1
1,user7,2018-11-01,DUB,147.500000,online,I2,NaN,2
2,user4,2018-11-02,TFS,24.049999,online,I2,NaN,3
3,user8,2018-10-29,MAD,59.709999,online,I2,NaN,4
4,user7,2018-11-01,LPA,37.299999,call center,I2,NaN,5
...,...,...,...,...,...,...,...,...
995,user2,2018-11-01,JMK,99.849998,online,I2,NaN,996
996,user10,2018-11-01,SVQ,34.610001,online,I2,NaN,997
997,user4,2018-10-30,MAD,49.880001,online,I2,NaN,998
998,user10,2018-11-02,CDG,152.960007,online,I2,NaN,999


In [13]:
dat_new = dat.fillna(value = modes, axis = 0)
dat_new

,user,booking_date,origin_airport,price,sales channel,company,user_country,index
0,user5,2018-11-01,MAD,58.200001,online,I2,Spain,1
1,user7,2018-11-01,DUB,147.500000,online,I2,NaN,2
2,user4,2018-11-02,TFS,24.049999,online,I2,NaN,3
3,user8,2018-10-29,MAD,59.709999,online,I2,NaN,4
4,user7,2018-11-01,LPA,37.299999,call center,I2,NaN,5
...,...,...,...,...,...,...,...,...
995,user2,2018-11-01,JMK,99.849998,online,I2,NaN,996
996,user10,2018-11-01,SVQ,34.610001,online,I2,NaN,997
997,user4,2018-10-30,MAD,49.880001,online,I2,NaN,998
998,user10,2018-11-02,CDG,152.960007,online,I2,NaN,999


In [14]:
dat_new.apply(lambda x: 100*np.sum(x.isna())/len(x))

user               0.0
booking_date       0.0
origin_airport     0.0
price              1.4
sales channel      0.0
company            0.0
user_country      98.9
index              0.0
dtype: float64

### Approach 2: Transform to New Category.

Missing values in a categorical variable can give you information, specially if they are not random. Therefore, it may be interesting to consider missing values as a new category instead of replacing them with the mode or some other value.

In [15]:
dat_new = dat.copy()
dat_new[categorical_variables] = dat[categorical_variables].fillna(value = 'UNKNOWN', axis = 0)
dat_new

,user,booking_date,origin_airport,price,sales channel,company,user_country,index
0,user5,2018-11-01,MAD,58.200001,online,I2,UNKNOWN,1
1,user7,2018-11-01,DUB,147.500000,online,I2,UNKNOWN,2
2,user4,2018-11-02,TFS,24.049999,online,I2,UNKNOWN,3
3,user8,2018-10-29,MAD,59.709999,online,I2,UNKNOWN,4
4,user7,2018-11-01,LPA,37.299999,call center,I2,UNKNOWN,5
...,...,...,...,...,...,...,...,...
995,user2,2018-11-01,JMK,99.849998,online,I2,UNKNOWN,996
996,user10,2018-11-01,SVQ,34.610001,online,I2,UNKNOWN,997
997,user4,2018-10-30,MAD,49.880001,online,I2,UNKNOWN,998
998,user10,2018-11-02,CDG,152.960007,online,I2,UNKNOWN,999


In [16]:
dat_new.apply(lambda x: 100*np.sum(x.isna())/len(x))

user              0.0
booking_date      0.0
origin_airport    0.0
price             1.4
sales channel     0.0
company           0.0
user_country      0.0
index             0.0
dtype: float64

## Numerical

### Approach 1: Replace with mean.

The next most basic approach regarding numerical variables will be to replace missing values with mean or median value.

In [18]:
means = dat[numerical_variables].mean()
means

price    81.592262
dtype: float64

Let's use this mean values to replace NAs.

In [20]:
dat_new = dat.fillna(means, axis = 1)
dat_new

NotImplementedError: Currently only can fill with dict/Series column by column

In [ ]:
dat_new.apply(lambda x: 100*np.sum(x.isna())/len(x))

user              0.0
booking_date      0.0
origin_airport    0.4
price             0.0
sales channel     0.0
ant               0.0
airline           0.0
dtype: float64

### Approach 2: Use an Algorithm.

Regarding filling missing values of numerical variables, there are much more complex methods to fill missing values, some of them even use machine learning models to try to predict this missing values.

This is the case of MICE: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074241/.

In [21]:
!pip install fancyimpute

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 892.6/892.6 kB 14.2 MB/s eta 0:00:00
     ---------------------------------------- 12.8/12.8 MB 7.4 MB/s eta 0:00:00
     ---------------------------------------- 324.5/324.5 kB ? eta 0:00:00
     ---------------------------------------- 154.7/154.7 kB ? eta 0:00:00
     ------------------------------------- 293.0/293.0 kB 17.7 MB/s eta 0:00:00
     ---------------------------------------- 72.0/72.0 kB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 8.2/8.2 MB 6.1 MB/s eta 0:00:00
  Using cached setuptools-68.2.2-py3-none-any.whl (807 kB)
  Using cached iniconfig-2.0.0-py3-none-any.whl (5.9 kB)
  Using cached pluggy-1.3.0-py3-none-any.whl (18 kB)
     ---------------------------------------- 85.1/85.1 kB 5.0 MB/s eta 0:00:00


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from fancyimpute import IterativeImputer as MICE # pip install fancyimpute

In [23]:
dat_new = dat.copy()
model = MICE()
dat_new[numerical_variables] = model.fit_transform(dat[numerical_variables])
dat_new

,user,booking_date,origin_airport,price,sales channel,company,user_country,index
0,user5,2018-11-01,MAD,58.200001,online,I2,NaN,1
1,user7,2018-11-01,DUB,147.500000,online,I2,NaN,2
2,user4,2018-11-02,TFS,24.049999,online,I2,NaN,3
3,user8,2018-10-29,MAD,59.709999,online,I2,NaN,4
4,user7,2018-11-01,LPA,37.299999,call center,I2,NaN,5
...,...,...,...,...,...,...,...,...
995,user2,2018-11-01,JMK,99.849998,online,I2,NaN,996
996,user10,2018-11-01,SVQ,34.610001,online,I2,NaN,997
997,user4,2018-10-30,MAD,49.880001,online,I2,NaN,998
998,user10,2018-11-02,CDG,152.960007,online,I2,NaN,999


In [24]:
dat_new.apply(lambda x: 100*np.sum(x.isna())/len(x))

user               0.0
booking_date       0.0
origin_airport     0.0
price              0.0
sales channel      0.0
company            0.0
user_country      99.0
index              0.0
dtype: float64

In [25]:
# from sklearn.linear_model import Lasso as model_constructor
from sklearn.linear_model import Lasso as model_constructor
dat_new = dat.copy()
model = MICE(estimator = model_constructor())
dat_new[numerical_variables] = model.fit_transform(dat[numerical_variables])
dat_new

,user,booking_date,origin_airport,price,sales channel,company,user_country,index
0,user5,2018-11-01,MAD,58.200001,online,I2,NaN,1
1,user7,2018-11-01,DUB,147.500000,online,I2,NaN,2
2,user4,2018-11-02,TFS,24.049999,online,I2,NaN,3
3,user8,2018-10-29,MAD,59.709999,online,I2,NaN,4
4,user7,2018-11-01,LPA,37.299999,call center,I2,NaN,5
...,...,...,...,...,...,...,...,...
995,user2,2018-11-01,JMK,99.849998,online,I2,NaN,996
996,user10,2018-11-01,SVQ,34.610001,online,I2,NaN,997
997,user4,2018-10-30,MAD,49.880001,online,I2,NaN,998
998,user10,2018-11-02,CDG,152.960007,online,I2,NaN,999


## Define Custom Function

Let's create our own custom function to fill missing values.

In [26]:
def fill_missing_values(X, num_method = 'mean', cat_method = 'new_category',
                estimator = None):
    categorical_variables = X.columns.values[dat.dtypes == 'object'].tolist()
    numerical_variables = X.columns.values[dat.dtypes == 'float64'].tolist()
    if num_method == 'mean':
        means = dat[numerical_variables].apply(lambda x: np.mean(x)).to_dict()
        X = X.fillna(value = means, axis = 0)

    elif num_method == 'mice':
        if estimator is None:
            X[numerical_variables] = MICE().fit_transform(X[numerical_variables])
        else:
            X[numerical_variables] = MICE(estimator = estimator).fit_transform(X[numerical_variables])
    if cat_method == 'new_category':
       X[categorical_variables] = X[categorical_variables].fillna(value = 'UNKNOWN', axis = 0)


    elif cat_method == 'mode':
        means = X[categorical_variables].apply(lambda x: np.mean(x)).to_dict()
        X = X.fillna(value = means, axis = 0)

    return X



In [27]:
new_dat = fill_missing_values(dat, "mean", "new_category")
new_dat

,user,booking_date,origin_airport,price,sales channel,company,user_country,index
0,user5,2018-11-01,MAD,58.200001,online,I2,UNKNOWN,1
1,user7,2018-11-01,DUB,147.500000,online,I2,UNKNOWN,2
2,user4,2018-11-02,TFS,24.049999,online,I2,UNKNOWN,3
3,user8,2018-10-29,MAD,59.709999,online,I2,UNKNOWN,4
4,user7,2018-11-01,LPA,37.299999,call center,I2,UNKNOWN,5
...,...,...,...,...,...,...,...,...
995,user2,2018-11-01,JMK,99.849998,online,I2,UNKNOWN,996
996,user10,2018-11-01,SVQ,34.610001,online,I2,UNKNOWN,997
997,user4,2018-10-30,MAD,49.880001,online,I2,UNKNOWN,998
998,user10,2018-11-02,CDG,152.960007,online,I2,UNKNOWN,999
